In [1]:
#| default_exp airsim

# Airsim Walkthrough
>  Documentation for using Microsoft Airsim in a Jupyter notebook
>  Airsim is a simulator for drones, cars and more.


In [2]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
# skip_showdoc: true to avoid running cells when rendering docs, and 
# skip_exec: true to skip this notebook when running tests. 
# this should be a raw cell 

In [1]:
assert False, "skip this cell"

AssertionError: skip this cell

In [3]:
#| hide
from fastcore.utils import *
import cv2
import numpy as np
from imutils import resize

import time

In [4]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *


In [5]:
from UAV.utils.display import *
from UAV.utils.sim_linux import *
from UAV.airsim_client import *
# import UAV.utils.sim_linux as sim

### Run a loop grabbing the camera

In [ ]:

import NH_path
import threading

rs = RunSim("AirSimNH", settings="config/settings_high_res.json")

asc = AirSimClient()

t = threading.Thread(target=NH_path.do_all, daemon=True)
t.start()

framecounter = 1
cam_num = 0
cams = ["high_res", "front_center", "front_right", "front_left", "bottom_center", "back_center"]
# with VideoWriter("images/airsim_test.mp4", 5.0) as video:

while(True):
    framecounter += 1
    state = asc.getMultirotorState()
    pos = state.kinematics_estimated.position
    img = asc.get_image(cams[cam_num], rgb2bgr=False)
    puttext(img, f"Frame: {framecounter} Pos: {pos.x_val:.2f}, {pos.y_val:.2f}, {pos.z_val:.2f}")
    img = resize(img, width=500)
    cv2.imshow("Camera", img)
    
    # video.add(img_bgr)
    k = cv2.waitKey(10)
    if k == ord('q') or k == ord('Q'):
        print("cancelLastTask")
        asc.cancelLastTask()
        # print(f"Landed state:  {state.landed_state}")
        if state.landed_state == 0:
            print("Landed state = 0,  so quiting")
            break  
    
    if k == ord('c') or k == ord('C'):
        cam_num += 1
        if cam_num >= len(cams):
            cam_num = 0
        print(f"Camera: {cams[cam_num]}")
        
    if k == 27:
        NH_path.stop()
        break
        
    # if framecounter > 50:
    #     break

asc.armDisarm(False)
asc.enableApiControl(False)
t.join(timeout=5)
cv2.destroyAllWindows()
rs.exit()


# video.show(width=500)

Starting Airsim  ['/home/jn/Airsim/AirSimNH/LinuxNoEditor/AirSimNH/Binaries/Linux/AirSimNH', '-ResX=800', '-ResY=600', '-windowed', '-settings=/home/jn/PycharmProjects/UAV/UAV/config/settings_high_res.json']
Started Airsim AirSimNH
Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)

Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)

This script is designed to fly on the streets of the Neighborhood environment
    and assumes the unreal position of the drone is [160, -1500, 120].
arming the drone...
taking off...
make sure we are hovering at 5 meters...
flying on path...
cancelLastTask
returning home...
cancelLastTask
landing...
disarming...
cancelLastTask
cancelLastTask


In [ ]:
#| hide
# from nbdev import nbdev_export
# nbdev_export()